In [ ]:

"""
Build an article-level master dataset by aggregating image features
and merging with the text/topics table.

Inputs:
  - articles_topics_bestk.xlsx         # text-level (one row per article)
  - Final_images_data.xlsx             # image-level (one row per image)
    required columns on image side: article_url, Core Figure Gender, Core Figure Skin, role
Outputs:
  - merged_row_level.xlsx              # row-level text×image merge (multi-rows per article if multi-images)
  - article_image_agg.xlsx             # image features aggregated to article level (one row per article that has images)
  - master_article_dataset.xlsx        # text table LEFT-JOIN with article-level image aggregates (one row per article)
"""

import os
import pandas as pd
from collections import Counter

# ---------- Paths ----------
TEXT_XLSX = "articles_topics_bestk.xlsx"
IMG_XLSX  = "Final_images_data.xlsx"

OUT_MERGED_ROW  = "merged_row_level.xlsx"
OUT_IMG_AGG     = "article_image_agg.xlsx"
OUT_MASTER      = "master_article_dataset.xlsx"

# ---------- Load ----------
text_df = pd.read_excel(TEXT_XLSX)
img_df  = pd.read_excel(IMG_XLSX)

# ---------- Key harmonization ----------
KEY = "article_url"
# coerce to string and strip
for df in (text_df, img_df):
    if KEY not in df.columns:
        raise ValueError(f"Missing key column '{KEY}' in: {df.shape}")
    df[KEY] = df[KEY].astype(str).str.strip()

# ---------- Basic diagnostics ----------
print("---- KEY diagnostics ----")
print("text key dtype:", text_df[KEY].dtype)
print("img  key dtype:", img_df[KEY].dtype)
print("text key nunique:", text_df[KEY].nunique())
print("img  key nunique:", img_df[KEY].nunique())
inter = set(text_df[KEY]).intersection(set(img_df[KEY]))
print("intersection size:", len(inter))
print("text-only keys   :", text_df[~text_df[KEY].isin(img_df[KEY])][KEY].nunique())
print("image-only keys  :", img_df[~img_df[KEY].isin(text_df[KEY])][KEY].nunique())

# ---------- Find/standardize image feature columns ----------
# Try to robustly locate columns (case/space tolerant)
def find_col(candidates, df):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    # fuzzy match
    for c in df.columns:
        if any(cand.lower() in c.lower() for cand in candidates):
            return c
    raise ValueError(f"Could not find any of {candidates} in image dataframe columns: {list(df.columns)[:10]} ...")

COL_GENDER = find_col(["Core Figure Gender", "core_figure_gender", "gender"], img_df)
COL_SKIN   = find_col(["Core Figure Skin", "core_figure_skin", "skin", "skintone"], img_df)
COL_ROLE   = find_col(["role", "core_role", "figure_role"], img_df)

print(f"[Image columns] gender='{COL_GENDER}', skin='{COL_SKIN}', role='{COL_ROLE}'")

# ---------- Clean & normalize codes as string (so counting is consistent) ----------
def norm_code(series):
    s = series.copy()
    # keep 99 or NA as '99' for "unknown"
    s = s.where(s.notna(), 99)
    # cast numerics to int safely; otherwise keep strings
    try:
        s = pd.to_numeric(s, errors="coerce").fillna(99).astype(int).astype(str)
    except Exception:
        s = s.astype(str).str.strip().replace({"": "99"})
    return s

img_df[COL_GENDER] = norm_code(img_df[COL_GENDER])
img_df[COL_SKIN]   = norm_code(img_df[COL_SKIN])
img_df[COL_ROLE]   = norm_code(img_df[COL_ROLE])

# ---------- Merge to get row-level joined table (optional but useful for audits) ----------
merged = text_df.merge(img_df[[KEY, COL_GENDER, COL_SKIN, COL_ROLE]], on=KEY, how="left")
print(f"Merged rows: {len(merged)} | articles: {text_df[KEY].nunique()}")
merged.to_excel(OUT_MERGED_ROW, index=False)
print(f"[OK] Saved row-level merge -> {OUT_MERGED_ROW}")

# ---------- Article-level aggregation of image features ----------
def dist_cols(prefix, counter, valid_keys=None):
    """Convert a Counter into wide columns with share and count.
       If valid_keys provided, only keep those codes explicitly (others go to 'other')."""
    total = sum(counter.values())
    out = {}
    if total == 0:
        return out
    keys = valid_keys or sorted(counter.keys())
    for k in keys:
        cnt = counter.get(k, 0)
        out[f"{prefix}{k}_cnt"]   = cnt
        out[f"{prefix}{k}_share"] = cnt / total
    # optional: collect unknown/other
    if valid_keys is not None:
        other_cnt = sum(v for kk, v in counter.items() if kk not in valid_keys)
        out[f"{prefix}other_cnt"]   = other_cnt
        out[f"{prefix}other_share"] = other_cnt / total
    return out

def aggregate_article(group):
    n_imgs = len(group)
    # distributions
    g_cnt = Counter(group[COL_GENDER])
    s_cnt = Counter(group[COL_SKIN])
    r_cnt = Counter(group[COL_ROLE])

    # majority labels (mode)
    maj_gender = max(g_cnt, key=g_cnt.get) if g_cnt else None
    maj_skin   = max(s_cnt, key=s_cnt.get) if s_cnt else None
    maj_role   = max(r_cnt, key=r_cnt.get) if r_cnt else None

    # shares dictionaries (limit to known codes to keep columns stable)
    # You can adjust the known code lists if your coding scheme differs.
    gender_out = dist_cols("gender_", g_cnt, valid_keys=["1","2","3","99"])
    skin_out   = dist_cols("skin_"  , s_cnt, valid_keys=["1","2","3","4","99"])
    role_out   = dist_cols("role_"  , r_cnt, valid_keys=["1","2","3","4","5","99"])

    # diversity flags
    skin_diverse = int(len([k for k in s_cnt if k not in ("99",)]) >= 2)  # ≥2 skin codes (excluding unknown)
    gender_mixed = int(len([k for k in g_cnt if k not in ("99",)]) >= 2)

    out = {
        KEY: group.iloc[0][KEY],
        "n_images": n_imgs,
        "maj_gender": maj_gender,
        "maj_skin": maj_skin,
        "maj_role": maj_role,
        "skin_diverse_flag": skin_diverse,
        "gender_mixed_flag": gender_mixed,
    }
    out.update(gender_out)
    out.update(skin_out)
    out.update(role_out)
    return pd.Series(out)

img_agg = merged.dropna(subset=[COL_GENDER, COL_SKIN, COL_ROLE], how="all") \
                .groupby(KEY, as_index=False).apply(aggregate_article)

img_agg.to_excel(OUT_IMG_AGG, index=False)
print(f"[OK] Saved image article aggregates -> {OUT_IMG_AGG}")

# ---------- Build master dataset (LEFT JOIN text with image aggregates) ----------
master = text_df.merge(img_agg, on=KEY, how="left")
master.to_excel(OUT_MASTER, index=False)
print(f"[OK] Saved master article dataset -> {OUT_MASTER}")

# ---------- Quick sanity prints ----------
print("\n[Master head]")
print(master[[KEY, "topic_label", "n_images", "maj_gender", "maj_skin", "maj_role"]].head())

print("\n[Null image aggregates by topic (top 10)]")
print(master[master["n_images"].isna()].groupby("topic_label").size().sort_values(ascending=False).head(10))

In [ ]:
import os
import re
import pandas as pd

# ---------------- Config: file paths ----------------
IMG_XLSX     = "final_images_data1.xlsx"       # image-level table
ARTICLE_XLSX = "stance_dual_by_year.xlsx"      # text-level table (sheet=article_level)

OUT_DIR   = "outputs_align_text_image"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_IMG_CLEAN   = os.path.join(OUT_DIR, "images_clean_labeled.xlsx")
OUT_IMG_AGG     = os.path.join(OUT_DIR, "article_image_agg.xlsx")
OUT_MASTER      = os.path.join(OUT_DIR, "master_article_merged.xlsx")
OUT_PIVOTS_XLSX = os.path.join(OUT_DIR, "topic_image_alignment_pivots.xlsx")

# ---------------- Helpers ----------------
def find_col(cols, patterns, required=True):
    """
    Fuzzy column finder: returns the first column whose lowercase name matches any regex pattern.
    """
    cl = [c for c in cols]
    lc = [c.lower() for c in cols]
    for pat in patterns:
        for i, name in enumerate(lc):
            if re.search(pat, name):
                return cl[i]
    if required:
        raise KeyError(f"Cannot find any column by patterns: {patterns}")
    return None

# ---------------- Read ----------------
img = pd.read_excel(IMG_XLSX)
txt = pd.read_excel(ARTICLE_XLSX, sheet_name="article_level")

# ---------------- Normalize join key ----------------
img_url_col = find_col(img.columns, [r"^article[_ ]?url$"])
txt_url_col = find_col(txt.columns, [r"^article[_ ]?url$"])
img[img_url_col] = img[img_url_col].astype(str).str.strip()
txt[txt_url_col] = txt[txt_url_col].astype(str).str.strip()

# ---------------- Locate image feature columns ----------------
col_age    = find_col(img.columns, [r"core.*age"])
col_gender = find_col(img.columns, [r"core.*gend"])
col_skin   = find_col(img.columns, [r"core.*skin"])
col_role   = find_col(img.columns, [r"^role$"])

# ---------------- Mappings ----------------
age_map = {1:"Child", 2:"Adult", 3:"Elderly", 99:"Uncertain"}
gender_map = {1:"Male", 2:"Female", 3:"Unclear", 99:"Unclear"}
skin_map = {1:"White", 2:"Black", 3:"Beige", 4:"Mixed/Unclear", 99:"Unclear"}
role_map = {1:"Elite", 2:"General public", 99:"Unclear/Abstract"}

# ---------------- Clean & map ----------------
def to_int_safe(s):
    try:
        return int(s)
    except Exception:
        return 99

img[col_age]    = img[col_age].apply(to_int_safe).map(age_map).fillna("Unclear")
img[col_gender] = img[col_gender].apply(to_int_safe).map(gender_map).fillna("Unclear")
img[col_skin]   = img[col_skin].apply(to_int_safe).map(skin_map).fillna("Unclear")
img[col_role]   = img[col_role].apply(to_int_safe).map(role_map).fillna("Unclear/Abstract")

img.to_excel(OUT_IMG_CLEAN, index=False)

# ---------------- Aggregate image features ----------------
def agg_share(df, key, cat_col, prefix):
    ct = (df.groupby([key, cat_col]).size().rename("count").reset_index())
    tot = ct.groupby(key)["count"].sum().rename("total").reset_index()
    ct = ct.merge(tot, on=key, how="left")
    ct["share"] = ct["count"] / ct["total"]
    w_count = ct.pivot(index=key, columns=cat_col, values="count").fillna(0)
    w_share = ct.pivot(index=key, columns=cat_col, values="share").fillna(0.0)
    w_count.columns = [f"{prefix}_{c}_count" for c in w_count.columns]
    w_share.columns = [f"{prefix}_{c}_share" for c in w_share.columns]
    wide = pd.concat([w_count, w_share], axis=1).reset_index()
    return wide

key = img_url_col
agg_gender = agg_share(img, key, col_gender, "gender")
agg_skin   = agg_share(img, key, col_skin,   "skin")
agg_role   = agg_share(img, key, col_role,   "role")
agg_age    = agg_share(img, key, col_age,    "age")

def dominant_label(df_wide, prefix):
    cnt_cols = [c for c in df_wide.columns if c.startswith(prefix) and c.endswith("_count")]
    lab = df_wide[cnt_cols].idxmax(axis=1).str.replace(f"{prefix}_", "", regex=False).str.replace("_count","", regex=False)
    return lab.rename(f"{prefix}_dominant")

for wide in [agg_gender, agg_skin, agg_role, agg_age]:
    prefix = wide.columns[1].split("_")[0]
    wide[prefix + "_dominant"] = dominant_label(wide, prefix)

img_agg = agg_gender.merge(agg_skin, on=key, how="outer") \
                    .merge(agg_role, on=key, how="outer") \
                    .merge(agg_age,   on=key, how="outer")
img_agg.to_excel(OUT_IMG_AGG, index=False)

# ---------------- Merge with text ----------------
topic_col = find_col(txt.columns, [r"topic[_ ]?label"], required=False)
sent_col  = find_col(txt.columns, [r"sent[_ ]?mean"], required=False)
title_col = find_col(txt.columns, [r"^title$"], required=False)

text_keep = [c for c in [txt_url_col, title_col, topic_col, sent_col] if c]
master = txt[text_keep].drop_duplicates()
master = master.merge(img_agg, left_on=txt_url_col, right_on=img_url_col, how="left")
master.to_excel(OUT_MASTER, index=False)

print("Done. Outputs saved in:", OUT_DIR)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

MASTER_XLSX = "outputs_align_text_image/master_article_merged.xlsx"

# ---------------- Load ----------------
df = pd.read_excel(MASTER_XLSX)

# Choose the topic column
topic_col = "topic_label"   # change if your text table uses another name

# ---------------- Visualization: stacked bar ----------------
def plot_stacked_bar(df, topic_col, prefix, top_n=15):
    """
    Make stacked bar chart: topic × category share
    """
    share_cols = [c for c in df.columns if c.startswith(prefix) and c.endswith("_share")]
    if not share_cols:
        print(f"No share cols found for prefix={prefix}")
        return
    
    # average share by topic
    mat = df.groupby(topic_col)[share_cols].mean()
    mat = mat[mat.sum(axis=1) > 0]   # drop empty topics
    
    # keep only top_n topics (by article count)
    topic_counts = df[topic_col].value_counts().head(top_n).index
    mat = mat.loc[mat.index.intersection(topic_counts)]
    
    # plot
    mat.plot(kind="bar", stacked=True, figsize=(12,6))
    plt.title(f"{prefix.capitalize()} distribution across topics")
    plt.ylabel("Average share")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

# ---------------- Visualization: heatmap ----------------
def plot_heatmap(df, topic_col, prefix, top_n=20):
    """
    Heatmap of topic × dominant label counts
    """
    dom_col = f"{prefix}_dominant"
    if dom_col not in df.columns:
        print(f"No dominant column found for {prefix}")
        return
    
    ct = pd.crosstab(df[topic_col], df[dom_col], normalize="index")
    
    # keep only top_n topics
    top_topics = df[topic_col].value_counts().head(top_n).index
    ct = ct.loc[ct.index.intersection(top_topics)]
    
    plt.figure(figsize=(10,6))
    plt.imshow(ct, aspect="auto", cmap="Blues")
    plt.colorbar(label="Share")
    plt.xticks(range(len(ct.columns)), ct.columns, rotation=45, ha="right")
    plt.yticks(range(len(ct.index)), ct.index)
    plt.title(f"Heatmap of {prefix} dominant labels across topics")
    plt.tight_layout()
    plt.show()

# ---------------- Run ----------------
for prefix in ["gender", "skin", "role"]:
    plot_stacked_bar(df, topic_col, prefix)
    plot_heatmap(df, topic_col, prefix)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# ========= Paths =========
TXT_XLSX = "stance_dual_by_year.xlsx"   # Must include sheet "article_level"
IMG_XLSX = "final_images_data1.xlsx"    # Must include article_url + role
OUT_DIR  = "multimodal_article_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# ========= Helpers =========
def role_code_to_label(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int,float)):
        x = int(x)
        return {1:"Elite",2:"General Public",99:"Unclear/Abstract"}.get(x,"Unclear/Abstract")
    s = str(x).lower()
    if "elite" in s: return "Elite"
    if "general" in s or "public" in s: return "General Public"
    if "abstract" in s or "unclear" in s: return "Unclear/Abstract"
    return "Unclear/Abstract"

def normalize_stance(s):
    s = str(s).strip().title()
    return {"Pos":"Positive","Neg":"Negative","Neu":"Neutral"}.get(s, s)

def majority(series):
    s = series.dropna()
    if s.empty: return np.nan
    return s.value_counts().idxmax()

def crosstab_and_plots(df, title_prefix, out_prefix):
    ct = pd.crosstab(df["stance_dual"], df["img_role"]).fillna(0).astype(int)
    ct.to_csv(os.path.join(OUT_DIR, f"{out_prefix}_crosstab_counts.csv"))

    # Heatmap
    mat = ct.values
    plt.figure(figsize=(6,4.6))
    ax = plt.gca()
    im = ax.imshow(mat, aspect="auto")
    ax.set_xticks(range(ct.shape[1])); ax.set_xticklabels(ct.columns, rotation=25, ha="right")
    ax.set_yticks(range(ct.shape[0])); ax.set_yticklabels(ct.index)
    for i in range(ct.shape[0]):
        for j in range(ct.shape[1]):
            ax.text(j, i, str(ct.iloc[i, j]), ha="center", va="center")
    plt.title(f"{title_prefix} — Crosstab (Counts)")
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{out_prefix}_counts_heatmap.png"), dpi=220)
    plt.close()

    # Row-percent stacked bar
    pct = ct.div(ct.sum(axis=1).replace(0,np.nan), axis=0).fillna(0)
    x = np.arange(len(pct.index))
    bottom = np.zeros(len(pct.index))
    plt.figure(figsize=(7.2,4.6))
    for col in pct.columns:
        plt.bar(x, pct[col].values, bottom=bottom, label=col)
        bottom += pct[col].values
    plt.xticks(x, pct.index, rotation=25, ha="right")
    plt.ylabel("Share")
    plt.title(f"{title_prefix} — Crosstab (Row %)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{out_prefix}_rowpct_stacked.png"), dpi=220)
    plt.close()

# ========= 1) Read TEXT (article_level sheet) =========
txt = pd.read_excel(TXT_XLSX, sheet_name="article_level")
txt["article_url"] = txt["article_url"].astype(str).str.strip()
txt["stance_dual"] = txt["stance_dual"].apply(normalize_stance)
txt["Year"] = pd.to_numeric(txt["Year"], errors="coerce").astype("Int64")

# ========= 2) Read IMAGES =========
img = pd.read_excel(IMG_XLSX, sheet_name=0)
img["article_url"] = img["article_url"].astype(str).str.strip()
img["img_role"] = img["role"].apply(role_code_to_label)

# ========= 3) Collapse images to article-level =========
img_article = img.groupby("article_url")["img_role"].agg(majority).reset_index()

# ========= 4) Merge TEXT + IMAGES =========
df_article = txt.merge(img_article, on="article_url", how="inner")

# Save article-level joined table
df_article.to_csv(os.path.join(OUT_DIR, "article_level_join.csv"), index=False)

# ========= 5) Global Crosstab =========
crosstab_and_plots(df_article, "Article-level (All)", "article_overall")

# ========= 6) By big_category (if available) =========
if "big_category" in df_article.columns:
    for bc, sub in df_article.groupby("big_category"):
        if sub.empty: continue
        safe = str(bc).replace("/","-").replace(" ","_")
        crosstab_and_plots(sub, f"Article-level — {bc}", f"article_bycat_{safe}")

print("Done. Outputs saved to:", OUT_DIR)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.parse import urlparse, urlunparse
from scipy.stats import chi2_contingency

# ========== Paths ==========
TXT_XLSX = "stance_dual_by_year.xlsx"   # must include sheet: "article_level"
IMG_XLSX = "final_images_data1.xlsx"    # must include: article_url + identity columns
OUT_DIR  = "multimodal_article_outputs_strict"
os.makedirs(OUT_DIR, exist_ok=True)

SIG_THR = 2.0  # threshold for standardized residual significance

# ========== Helpers ==========
def normalize_stance(s):
    s = str(s).strip().title()
    return {"Pos": "Positive", "Neg": "Negative", "Neu": "Neutral"}.get(s, s)

def _to_int_if_str_digit(x):
    try:
        if isinstance(x, str) and x.strip().isdigit():
            return int(x.strip())
    except Exception:
        pass
    return x

def role_code_to_label(x):
    """
    Role coding: 1=Elite; 2=General Public; 99=Unclear/Abstract
    """
    x = _to_int_if_str_digit(x)
    if pd.isna(x):
        return "Unclear/Abstract"
    if isinstance(x, (int, float)) and np.isfinite(x):
        return {1: "Elite", 2: "General Public", 99: "Unclear/Abstract"}.get(int(x), "Unclear/Abstract")
    s = str(x).strip().lower()
    if "elite" in s: return "Elite"
    if "general" in s or "public" in s: return "General Public"
    return "Unclear/Abstract"

def gender_code_to_label(x):
    """
    Core Figure Gender: 1=Male; 2=Female; 3=Unclear
    """
    x = _to_int_if_str_digit(x)
    if pd.isna(x):
        return "Unclear"
    if isinstance(x, (int, float)) and np.isfinite(x):
        return {1: "Male", 2: "Female", 3: "Unclear"}.get(int(x), "Unclear")
    s = str(x).strip().lower()
    if "male" in s or "man" in s: return "Male"
    if "female" in s or "woman" in s: return "Female"
    return "Unclear"

def skin_code_to_label(x):
    """
    Core Figure skin: 1=White; 2=Black; 3=Beige (also 'Begie'); 4=Mixed/Unclear
    """
    x = _to_int_if_str_digit(x)
    if pd.isna(x):
        return "Unclear"
    if isinstance(x, (int, float)) and np.isfinite(x):
        return {1: "White", 2: "Black", 3: "Beige", 4: "Mixed/Unclear"}.get(int(x), "Unclear")
    s = str(x).strip().lower()
    if "white" in s: return "White"
    if "black" in s: return "Black"
    if "beige" in s or "begie" in s: return "Beige"  # fix common misspelling
    if "mixed" in s: return "Mixed/Unclear"
    return "Unclear"

def normalize_url(u, drop_query=True, drop_fragment=True):
    """Standardize URL for joining."""
    if pd.isna(u):
        return np.nan
    u = str(u).strip()
    if not u:
        return np.nan
    try:
        p = urlparse(u)
        scheme = (p.scheme or "http").lower()
        netloc = p.netloc.lower()
        path = p.path.rstrip("/")
        query = "" if drop_query else p.query
        fragment = "" if drop_fragment else p.fragment
        return urlunparse((scheme, netloc, path, p.params, query, fragment))
    except Exception:
        return u.strip().lower().rstrip("/")

def majority(series):
    s = series.dropna()
    if s.empty:
        return np.nan
    return s.value_counts().idxmax()

def find_col(df, candidates):
    """Case-insensitive exact match; return original column name or None."""
    cand_norm = [str(x).strip().lower() for x in candidates]
    for col in df.columns:
        if str(col).strip().lower() in cand_norm:
            return col
    return None

# ========== Chi-square and residuals ==========
def chi2_with_std_resid(ct: pd.DataFrame):
    """Return chi2, p, dof, expected_df, std_resid_df, Cramer's V."""
    ct = ct.astype(float)
    n = ct.values.sum()
    if n == 0:
        expected_df = pd.DataFrame(0, index=ct.index, columns=ct.columns)
        std_resid = pd.DataFrame(np.nan, index=ct.index, columns=ct.columns)
        return 0.0, 1.0, (ct.shape[0]-1)*(ct.shape[1]-1), expected_df, std_resid, 0.0

    chi2, p, dof, expected = chi2_contingency(ct.values, correction=False)
    expected_df = pd.DataFrame(expected, index=ct.index, columns=ct.columns)

    row_p = ct.sum(axis=1) / n
    col_p = ct.sum(axis=0) / n
    denom = np.sqrt(expected_df * (1 - row_p.values[:, None]) * (1 - col_p.values[None, :]))
    std_resid = (ct - expected_df) / denom.replace(0, np.nan)

    r, c = ct.shape
    cramers_v = np.sqrt(chi2 / (n * (min(r - 1, c - 1)))) if min(r - 1, c - 1) > 0 else 0.0
    return chi2, p, dof, expected_df, std_resid, cramers_v

def plot_crosstab_full(df, by_col, title_prefix, out_prefix, out_dir, sig_thr=2.0):
    """
    Build Sentiment × by_col crosstab; export tables and three plots; export candidate URLs.
    """
    # contingency
    ct = pd.crosstab(df["stance_dual"], df[by_col]).fillna(0).astype(int)
    chi2, p, dof, expected, std_resid, cramers_v = chi2_with_std_resid(ct)
    pct = ct.div(ct.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

    # export tables
    xlsx_path = os.path.join(out_dir, f"{out_prefix}_crosstab.xlsx")
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as w:
        ct.to_excel(w, "counts")
        pct.to_excel(w, "row_pct")
        expected.round(2).to_excel(w, "expected")
        std_resid.round(3).to_excel(w, "std_resid")
        (std_resid.abs() >= sig_thr).astype(int).to_excel(w, f"sig_mask_abs>={sig_thr:g}")

    # counts heatmap
    plt.figure(figsize=(6, 4.6))
    ax = plt.gca()
    im = ax.imshow(ct.values, aspect="auto")
    ax.set_xticks(range(ct.shape[1])); ax.set_xticklabels(ct.columns, rotation=25, ha="right")
    ax.set_yticks(range(ct.shape[0])); ax.set_yticklabels(ct.index)
    for i in range(ct.shape[0]):
        for j in range(ct.shape[1]):
            ax.text(j, i, str(ct.iloc[i, j]), ha="center", va="center")
    plt.title(f"{title_prefix} — Crosstab (Counts)")
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{out_prefix}_counts_heatmap.png"), dpi=220)
    plt.close()

    # row-% stacked bars
    x = np.arange(len(pct.index))
    bottom = np.zeros(len(pct.index))
    plt.figure(figsize=(7.2, 4.6))
    for col in pct.columns:
        plt.bar(x, pct[col].values, bottom=bottom, label=col)
        bottom += pct[col].values
    plt.xticks(x, pct.index, rotation=25, ha="right")
    plt.ylabel("Share")
    plt.title(f"{title_prefix} — Crosstab (Row %)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{out_prefix}_rowpct_stacked.png"), dpi=220)
    plt.close()

    # standardized residuals heatmap with significance boxes
    import matplotlib.patches as patches
    vmax = float(np.nanmax(np.abs(std_resid.values))) if std_resid.size else 1.0
    plt.figure(figsize=(6.6, 4.8))
    ax = plt.gca()
    im = ax.imshow(std_resid.values, vmin=-vmax, vmax=vmax, aspect="auto")
    ax.set_xticks(range(std_resid.shape[1])); ax.set_xticklabels(std_resid.columns, rotation=25, ha="right")
    ax.set_yticks(range(std_resid.shape[0])); ax.set_yticklabels(std_resid.index)
    for i in range(std_resid.shape[0]):
        for j in range(std_resid.shape[1]):
            val = std_resid.iloc[i, j]
            ax.text(j, i, ("" if pd.isna(val) else f"{val:.2f}"), ha="center", va="center", fontsize=10)
            if pd.notna(val) and abs(val) >= sig_thr:
                ax.add_patch(patches.Rectangle((j - 0.5, i - 0.5), 1, 1, linewidth=2,
                                               edgecolor="black", facecolor="none"))
    plt.title(f"{title_prefix} — Std. Residuals (chi2={chi2:.2f}, p={p:.2e}, V={cramers_v:.3f}; thr={sig_thr:g})")
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("Standardized Residual")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{out_prefix}_std_resid_heatmap.png"), dpi=220)
    plt.close()

    # export candidate URLs (over-represented and under-represented cells)
    res_map = std_resid.stack().to_dict()
    df_tmp = df.loc[:, ["article_url", "stance_dual", by_col]].dropna()
    df_tmp["std_resid"] = df_tmp.apply(lambda r: res_map.get((r["stance_dual"], r[by_col])), axis=1)
    over  = df_tmp[df_tmp["std_resid"] >=  sig_thr].copy()
    under = df_tmp[df_tmp["std_resid"] <= -sig_thr].copy()

    cand_path = os.path.join(out_dir, f"{out_prefix}_candidates.xlsx")
    with pd.ExcelWriter(cand_path, engine="openpyxl") as w:
        over.sort_values("std_resid", ascending=False).to_excel(w, "over_rep", index=False)
        under.sort_values("std_resid", ascending=True).to_excel(w, "under_rep", index=False)

    return {"chi2": chi2, "p": p, "cramers_v": cramers_v,
            "counts": ct, "row_pct": pct, "std_resid": std_resid}

# Unified export for role/gender/skin candidates (including tension for role)
def export_unified_candidates(df_article, out_path, sig_thr=2.0):
    writer = pd.ExcelWriter(out_path, engine="openpyxl")

    def residual_base(by_col):
        ct = pd.crosstab(df_article["stance_dual"], df_article[by_col]).fillna(0).astype(int)
        if ct.empty or ct.sum().sum() == 0:
            return None, None
        _, _, _, _, std_resid, _ = chi2_with_std_resid(ct.astype(float))
        mapping = std_resid.stack().to_dict()
        base = df_article.loc[:, ["article_url", "stance_dual", by_col]].dropna()
        base["std_resid"] = base.apply(lambda r: mapping.get((r["stance_dual"], r[by_col])), axis=1)
        return base, std_resid

    # Role: synergy, absence, tension
    base, std_role = residual_base("img_role")
    if base is not None:
        synergy_pairs = {("Positive", "Elite"), ("Negative", "General Public")}
        tension_pairs = {("Positive", "General Public"), ("Negative", "Elite")}
        role_synergy = base[(base["std_resid"] >= sig_thr) &
                            (base.apply(lambda r: (r["stance_dual"], r["img_role"]) in synergy_pairs, axis=1))].copy()
        role_absence = base[(base["std_resid"] <= -sig_thr)].copy()
        role_tension = base[base.apply(lambda r: (r["stance_dual"], r["img_role"]) in tension_pairs, axis=1)].copy()
        role_tension = role_tension.sort_values("std_resid", key=lambda s: s.abs(), ascending=False)

        role_synergy.sort_values("std_resid", ascending=False).to_excel(writer, "role_synergy", index=False)
        role_absence.sort_values("std_resid", ascending=True).to_excel(writer, "role_absence", index=False)
        role_tension.to_excel(writer, "role_tension", index=False)
        if std_role is not None:
            std_role.round(3).to_excel(writer, "role_std_resid")

    # Gender: synergy (positive residual) and absence (negative residual)
    if "img_gender" in df_article.columns:
        base, std_gender = residual_base("img_gender")
        if base is not None:
            g_syn = base[base["std_resid"] >= sig_thr].copy()
            g_abs = base[base["std_resid"] <= -sig_thr].copy()
            g_syn.sort_values("std_resid", ascending=False).to_excel(writer, "gender_synergy", index=False)
            g_abs.sort_values("std_resid", ascending=True).to_excel(writer, "gender_absence", index=False)
            if std_gender is not None:
                std_gender.round(3).to_excel(writer, "gender_std_resid")

    # Skin: synergy and absence
    if "img_skin" in df_article.columns:
        base, std_skin = residual_base("img_skin")
        if base is not None:
            s_syn = base[base["std_resid"] >= sig_thr].copy()
            s_abs = base[base["std_resid"] <= -sig_thr].copy()
            s_syn.sort_values("std_resid", ascending=False).to_excel(writer, "skin_synergy", index=False)
            s_abs.sort_values("std_resid", ascending=True).to_excel(writer, "skin_absence", index=False)
            if std_skin is not None:
                std_skin.round(3).to_excel(writer, "skin_std_resid")

    writer.close()

# ========== 1) Read and normalize text ==========
txt = pd.read_excel(TXT_XLSX, sheet_name="article_level")
assert "article_url" in txt.columns, "Text sheet 'article_level' must include article_url"

txt["article_url_raw"] = txt["article_url"]
txt["article_url"] = txt["article_url"].apply(lambda x: normalize_url(x, drop_query=True, drop_fragment=True))
txt = txt.dropna(subset=["article_url"]).copy()

if "stance_dual" not in txt.columns:
    raise ValueError("Text sheet must include 'stance_dual'")
txt["stance_dual"] = txt["stance_dual"].apply(normalize_stance)

if "Year" in txt.columns:
    txt["Year"] = pd.to_numeric(txt["Year"], errors="coerce").astype("Int64")
else:
    txt["Year"] = pd.NA

txt_agg = (
    txt.groupby("article_url")
       .agg(
           stance_dual=("stance_dual", majority),
           Year=("Year", majority),
           big_category=("big_category", majority) if "big_category" in txt.columns else ("Year", lambda s: "All")
       )
       .reset_index()
)

# ========== 2) Read and normalize image sheet ==========
img = pd.read_excel(IMG_XLSX, sheet_name=0)
assert "article_url" in img.columns, "Image sheet must include article_url"
img.columns = [str(c).strip() for c in img.columns]  # trim header whitespace

# detect columns (case-insensitive), including your exact names
role_col   = find_col(img, ["img_role", "role", "main_role", "Role", "Core Figure Role"])
gender_col = find_col(img, ["Core Figure Gender", "img_gender", "gender", "Gender",
                            "gender_dom", "gender_dominant", "main_gender"])
skin_col   = find_col(img, ["Core Figure skin", "Core Figure Skin", "img_skin", "skin", "Skin",
                            "skin_dom", "skin_dominant", "skin_tone", "skin_tone_dom"])
if role_col is None:
    raise ValueError("Image sheet must include a role column (e.g., 'Core Figure Role' or 'role').")

img["article_url_raw"] = img["article_url"]
img["article_url"] = img["article_url"].apply(lambda x: normalize_url(x, drop_query=True, drop_fragment=True))
img = img.dropna(subset=["article_url"]).copy()

img["img_role"] = img[role_col].apply(role_code_to_label)
if gender_col is not None:
    img["img_gender"] = img[gender_col].apply(gender_code_to_label)
if skin_col is not None:
    img["img_skin"] = img[skin_col].apply(skin_code_to_label)

# aggregate to one row per article by majority vote
agg_dict = {"img_role": ("img_role", majority)}
if "img_gender" in img.columns:
    agg_dict["img_gender"] = ("img_gender", majority)
if "img_skin" in img.columns:
    agg_dict["img_skin"] = ("img_skin", majority)

img_agg = img.groupby("article_url").agg(**agg_dict).reset_index()

# quick check
print("Detected columns -> role_col =", role_col, "| gender_col =", gender_col, "| skin_col =", skin_col)

# ========== 3) Intersect URLs and export unmatched lists ==========
text_urls  = set(txt_agg["article_url"].unique())
image_urls = set(img_agg["article_url"].unique())
intersect  = text_urls & image_urls

pd.DataFrame({"text_only_urls": sorted(text_urls - image_urls)}).to_excel(
    os.path.join(OUT_DIR, "unmatched_text_only_urls.xlsx"), index=False
)
pd.DataFrame({"image_only_urls": sorted(image_urls - text_urls)}).to_excel(
    os.path.join(OUT_DIR, "unmatched_image_only_urls.xlsx"), index=False
)

# keep intersection
txt_keep = txt_agg[txt_agg["article_url"].isin(intersect)].copy()
img_keep = img_agg[img_agg["article_url"].isin(intersect)].copy()

# ========== 4) Strict inner join ==========
df_article = txt_keep.merge(img_keep, on="article_url", how="inner")
assert df_article["article_url"].is_unique, "Merged article_url should be unique per row"

# save joined table
article_xlsx = os.path.join(OUT_DIR, "article_level_join_strict.xlsx")
df_article.to_excel(article_xlsx, index=False)

# ========== 5) Crosstabs and visualizations ==========
stance_order = ["Positive", "Negative", "Neutral"]
role_order   = ["Elite", "General Public", "Unclear/Abstract"]
gender_order = ["Male", "Female", "Unclear"]
skin_order   = ["White", "Black", "Beige", "Mixed/Unclear", "Unclear"]

df_article["stance_dual"] = pd.Categorical(df_article["stance_dual"], categories=stance_order, ordered=False)
df_article["img_role"]    = pd.Categorical(df_article["img_role"],    categories=role_order,   ordered=False)
if "img_gender" in df_article.columns:
    df_article["img_gender"] = pd.Categorical(df_article["img_gender"], categories=gender_order, ordered=False)
if "img_skin" in df_article.columns:
    df_article["img_skin"]   = pd.Categorical(df_article["img_skin"],   categories=skin_order,   ordered=False)

# Overall: Sentiment × Role/Gender/Skin
plot_crosstab_full(df_article, "img_role",
                   "Article-level (Strict) — Sentiment × Role",
                   "article_overall_strict_role", OUT_DIR, sig_thr=SIG_THR)

if "img_gender" in df_article.columns:
    plot_crosstab_full(df_article, "img_gender",
                       "Article-level (Strict) — Sentiment × Gender",
                       "article_overall_strict_gender", OUT_DIR, sig_thr=SIG_THR)

if "img_skin" in df_article.columns:
    plot_crosstab_full(df_article, "img_skin",
                       "Article-level (Strict) — Sentiment × Skin",
                       "article_overall_strict_skin", OUT_DIR, sig_thr=SIG_THR)

# By big_category (if present)
if "big_category" in df_article.columns:
    for bc, sub in df_article.groupby("big_category"):
        if sub.empty:
            continue
        safe = str(bc).replace("/", "-").replace("\\", "-").replace(" ", "_")
        plot_crosstab_full(sub, "img_role",
                           f"Article-level — {bc} (Strict) — S×Role",
                           f"article_bycat_{safe}_strict_role", OUT_DIR, sig_thr=SIG_THR)
        if "img_gender" in sub.columns:
            plot_crosstab_full(sub, "img_gender",
                               f"Article-level — {bc} (Strict) — S×Gender",
                               f"article_bycat_{safe}_strict_gender", OUT_DIR, sig_thr=SIG_THR)
        if "img_skin" in sub.columns:
            plot_crosstab_full(sub, "img_skin",
                               f"Article-level — {bc} (Strict) — S×Skin",
                               f"article_bycat_{safe}_strict_skin", OUT_DIR, sig_thr=SIG_THR)

# ========== 6) Unified candidates export ==========
export_unified_candidates(
    df_article,
    os.path.join(OUT_DIR, "sentiment_identity_candidates_unified.xlsx"),
    sig_thr=SIG_THR
)

# ========== 7) Quick summary ==========
with open(os.path.join(OUT_DIR, "summary_stats.txt"), "w", encoding="utf-8") as f:
    f.write(f"Text raw rows: {len(txt)}\n")
    f.write(f"Text unique URLs: {txt_agg['article_url'].nunique()}\n")
    f.write(f"Image raw rows: {len(img)}\n")
    f.write(f"Image unique URLs: {img_agg['article_url'].nunique()}\n")
    f.write(f"Intersection URLs: {len(intersect)}\n")
    f.write(f"Final joined rows (strict 1:1): {len(df_article)}\n")

print("Done. Outputs in:", OUT_DIR)
print("XLSX files written:",
      article_xlsx,
      os.path.join(OUT_DIR, "unmatched_text_only_urls.xlsx"),
      os.path.join(OUT_DIR, "unmatched_image_only_urls.xlsx"),
      os.path.join(OUT_DIR, "sentiment_identity_candidates_unified.xlsx"))